In [0]:
%sql
CREATE TABLE IF NOT EXISTS projetos.silver.ecommerce_transactions (
  id BIGINT GENERATED ALWAYS AS IDENTITY PRIMARY KEY COMMENT 'Unique identifier for the transaction record.',
  event_date DATE NOT NULL COMMENT 'Date of the transaction, used for partitioning.',
  event_timestamp TIMESTAMP COMMENT 'Full timestamp of the transaction event.',
  invoice_no STRING COMMENT 'Identifier for the invoice, linking multiple items to a single sale.',
  stock_code STRING COMMENT 'Product identifier code (SKU).',
  description STRING COMMENT 'Product description.',
  quantity BIGINT COMMENT 'Number of units for the specific product. Can be negative for returns.',
  unit_price DOUBLE COMMENT 'Price per one unit of the product.',
  customer_id BIGINT COMMENT 'Identifier for the customer.',
  country STRING COMMENT 'Country where the transaction occurred.',
  ingestion_timestamp TIMESTAMP_NTZ DEFAULT CURRENT_TIMESTAMP() COMMENT 'Timestamp when the record was processed into the silver layer.',
  hash VARCHAR(32) COMMENT 'MD5 hash of the row content to track changes from the source.',
  data_source VARCHAR(1024) COMMENT 'Source system or file path where the data originated.',
  pipeline_run_id VARCHAR(255) COMMENT 'Unique ID of the pipeline run that processed this record.'
)
USING DELTA
PARTITIONED BY (event_date)
COMMENT 'Cleaned and enriched table of individual e-commerce transactions. This is the single source of truth for all sales line items.'
TBLPROPERTIES('delta.feature.allowColumnDefaults' = 'supported')

In [0]:
%sql
CREATE TABLE IF NOT EXISTS projetos.gold.ecommerce_sales_agg (
  id BIGINT GENERATED ALWAYS AS IDENTITY PRIMARY KEY COMMENT 'Unique identifier for the aggregated record.',
  event_date DATE NOT NULL COMMENT 'Date of the sales event, used for partitioning.',
  event_timestamp TIMESTAMP COMMENT 'Timestamp of the first event associated with the invoice.',
  customer_id BIGINT COMMENT 'Identifier for the customer.',
  invoice_no STRING COMMENT 'Unique identifier for the invoice or sale.',
  items_carts BIGINT COMMENT 'Total number of items included in the invoice.',
  cart_price DECIMAL(18, 2) COMMENT 'Total price for all items in the invoice.',
  country STRING COMMENT 'Country where the sale occurred.'
)
USING DELTA
PARTITIONED BY (event_date)
COMMENT 'Aggregated daily sales data, summarized by invoice. This is the primary table for BI dashboards and sales analytics.'